In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = './data/'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition
!unzip -q train.zip -d . 


  0%|          | 0.00/814M [00:00<?, ?B/s]
  0%|          | 1.00M/814M [00:00<10:30, 1.35MB/s]
  0%|          | 2.00M/814M [00:00<05:16, 2.69MB/s]
  0%|          | 4.00M/814M [00:01<02:29, 5.67MB/s]
  1%|          | 8.00M/814M [00:01<01:09, 12.2MB/s]
  1%|          | 10.0M/814M [00:01<01:08, 12.4MB/s]
  2%|▏         | 13.0M/814M [00:01<00:52, 16.1MB/s]
  2%|▏         | 17.0M/814M [00:01<00:40, 20.7MB/s]
  2%|▏         | 20.0M/814M [00:01<00:37, 22.0MB/s]
  3%|▎         | 23.0M/814M [00:01<00:37, 22.4MB/s]
  3%|▎         | 27.0M/814M [00:01<00:31, 26.0MB/s]
  4%|▎         | 30.0M/814M [00:02<00:30, 26.7MB/s]
  4%|▍         | 33.0M/814M [00:02<00:29, 27.8MB/s]
  4%|▍         | 36.0M/814M [00:02<00:30, 26.7MB/s]
  5%|▍         | 39.0M/814M [00:02<00:29, 27.9MB/s]
  5%|▌         | 42.0M/814M [00:02<00:30, 26.1MB/s]
  6%|▌         | 45.0M/814M [00:02<00:29, 27.2MB/s]
  6%|▌         | 48.0M/814M [00:02<00:28, 28.0MB/s]
  6%|▋         | 51.0M/814M [00:02<00:27, 28.7MB/s]
  7%|▋         | 55.

In [2]:
import os 
import tensorflow as tf 
import shutil

relative_path = 'C:/Users/sou05094/Desktop/dataset'
relative_path1 = 'C:/Users/sou05094/Desktop/data'
#os.mkdir(relative_path1)
#os.mkdir(os.path.join(relative_path, 'cat'))
#os.mkdir(os.path.join(relative_path, 'dog'))

for i in os.listdir(relative_path + '/train/train/'):
  #if 'cat' in i:
   # shutil.copyfile( relative_path + '/train/train/' + i , relative_path1 + '/cat/' + i )
  if 'dog' in i:
    shutil.copyfile( relative_path + '/train/train/' + i , relative_path1 + '/dog/' + i)



Found 37500 files belonging to 4 classes.
Using 30000 files for training.
Found 37500 files belonging to 4 classes.
Using 7500 files for validation.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [12]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'C:/Users/sou05094/Desktop/data/',
  image_size=(150,150),
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'C:/Users/sou05094/Desktop/data/',
  image_size=(150,150),
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

print(train_ds)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [3]:
def 전처리함수(i, 정답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

In [4]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

# weight파일 (inception_v3 모델의 w값)
open('inception_v3.h5', 'wb').write(r.content)

87910968

In [9]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
# 남이 만든건 input shape 조심 할 것
# 마지막 레이어 출력 여부 include_top
inception_model = InceptionV3(input_shape = (150,150,3), include_top = False, weights = None)
inception_model.load_weights('inception_v3.h5')
#inception_model.summary()
# 학습금지 레이어 설정
for i in inception_model.layers:
    # w값 고정
    i.trainable = False

lastlayer = inception_model.get_layer('mixed7')

print(lastlayer)
print(lastlayer.output)
print(lastlayer.output_shape)

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")
(None, 7, 7, 768)


In [ ]:
# fine tuning 하는 법
unfreeze = False
for i in inception_model.layers:
    # w값 고정
    if i.name == 'mixed6':
        unfreeze = True
    if unfreeze == True:
        i.trainable = True
         
# weight들이 아주 조금씩 업데이트 됨 (lr = 0.00001)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr = 0.00001), metrics=['acc'])
model.fit(train_ds, validation_data=val_ds, epochs=2)

In [13]:
import tensorflow as tf

# 레이어 연결
layer1 = tf.keras.layers.Flatten()(lastlayer.output)
layer2 = tf.keras.layers.Dense(1024,activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

model = tf.keras.Model(inception_model.input ,layer3)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2
313/313 [==============================] - 213s 674ms/step - loss: 18.4259 - acc: 0.5875 - val_loss: 0.6534 - val_acc: 0.6536
Epoch 2/2
313/313 [==============================] - 203s 649ms/step - loss: 0.6406 - acc: 0.6460 - val_loss: 0.6208 - val_acc: 0.6876
